In [1]:
# imports

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

### Step 1: Establish broad API call
* ensures API ok


In [6]:
import json

import requests
import os

FS_API_KEY = os.getenv('FOURSQUARE_API_KEY')


print(FS_API_KEY)

location = "Vancouver, Canada"
url = "https://api.foursquare.com/v3/places/search?near=" + location

# Create dictionary for headers

payload = {}
headers = {
        "Accept": "application/json",
        'Authorization': FS_API_KEY
            }

response = requests.request("GET", url, headers=headers, data=payload)

print(response.json())

fsq3IfwEJ3A+/5rg/MDnHEO7RgiDAxoRIW9j7zW/C0ScVlE=
{'results': [{'fsq_id': '4bdf6cddffdec9287a09eca1', 'categories': [{'id': 16039, 'name': 'Urban Park', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_', 'suffix': '.png'}}], 'chains': [], 'distance': 998, 'geocodes': {'main': {'latitude': 49.241325, 'longitude': -123.11111}, 'roof': {'latitude': 49.236986, 'longitude': -123.110884}}, 'link': '/v3/places/4bdf6cddffdec9287a09eca1', 'location': {'address': '4600 Cambie St', 'country': 'CA', 'formatted_address': '4600 Cambie St, Vancouver BC V5Z 2Z1', 'locality': 'Vancouver', 'postcode': 'V5Z 2Z1', 'region': 'BC'}, 'name': 'Queen Elizabeth Park', 'related_places': {'children': [{'fsq_id': '4e07cfdfb61c60b04547fd16', 'name': 'Queen Elizabeth Parking Lot'}, {'fsq_id': '4e5957dd52b1b78b819bf083', 'name': 'Celebration Pavilion'}, {'fsq_id': '637ab2e6da100869764d3264', 'name': 'Quarry Gardens'}, {'fsq_id': '4aac6667f964a520d85d20e3', 'name': 'Bloedel Floral Conserv

## Step 2: Prepare and Test Lat Long info from SQL
* ensure syntax is correct
* update Postgres DB to avoid potential concatenation complication in code.
* 

In [3]:
## SQL to create a new column in the City_bike Table. 
# Performed in Postgres

'''UPDATE city_bikes
SET lat_long = (select concat(latitude,',',longitude));'''



"UPDATE city_bikes\nSET lat_long = (select concat(latitude,',',longitude));"

## Step 3: Create DataFrame of Lat/Long and iterate to create API calls

In [48]:
import pandas as pd

data = pd.read_csv('../data/city_bikes.csv')
vancouver_stations = pd.DataFrame(data)
vancouver_stations


,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra_ebikes,extra_has_ebikes,extra_last_updated,lat_long
0,5,29,7a19c49f486d7c0c02b3685d7b240448,49.262487,-123.114397,10th & Cambie,2023-09-21 19:03:51.469000,6,True,1695322925,"49.262487,-123.114397"
1,12,4,32603a87cfca71d0f7dfa3513bad69d5,49.274566,-123.121817,Yaletown-Roundhouse Station,2023-09-21 19:03:51.456000,0,True,1695322945,"49.274566,-123.121817"
2,13,13,6d42fa40360f9a6b2bf641c7b8bb2862,49.279764,-123.110154,Dunsmuir & Beatty,2023-09-21 19:03:51.457000,1,True,1695322662,"49.279764,-123.110154"
3,11,5,66f873d641d448bd1572ab086665a458,49.260599,-123.113504,12th & Yukon (City Hall),2023-09-21 19:03:51.458000,4,True,1695322780,"49.260599,-123.113504"
4,9,7,485d4d24c803cfde829ab89699fed833,49.264215,-123.117772,8th & Ash,2023-09-21 19:03:51.477000,4,True,1695322949,"49.264215,-123.117772"
...,...,...,...,...,...,...,...,...,...,...,...
240,34,6,a74744ce4bb7ea2aa9f406ac8bff95d8,49.280977,-123.035969,PNE - Hastings & Windermere,2023-09-21 19:03:51.275000,2,True,1695322761,"49.280977,-123.035969"
241,8,11,5699b40126e10c2f68eefc3eb18ff3a1,49.270783,-123.141564,1st & Fir,2023-09-21 19:03:51.247000,1,True,1695322960,"49.270783,-123.141564"
242,15,7,34fd37d12eb989b49518ef53941ee3ff,49.264019,-123.209176,Sasamat & 10th,2023-09-21 19:03:51.199000,3,True,1695322610,"49.264019,-123.209176"
243,18,2,b7dd37a7dd668d6d10024b7f18acc438,49.265800,-123.205960,Trimble & 8th,2023-09-21 19:03:51.246000,1,True,1695322940,"49.2658,-123.20596"


In [7]:
from pprint import pprint
radius = 1000
def foursquare_info(lat_long, radius):
    API_results = []
    category_filter = "11062,13003,13004,13005,13006,13007,13008,13009,13010,13011,13012,13013,13014,13015,13016,13017,13018,13019,13020,13021,13022,13023,13024,13025,13059" 
    fields = "name,categories,popularity,rating,price,distance,fsq_id"
    for row in lat_longs:
        url = f"https://api.foursquare.com/v3/places/search?ll={row}&radius={radius}&fields={fields}&categories={category_filter}"
        # Create dictionary for headers
        payload = {}
        headers = {
                "Accept": "application/json",
                'Authorization': FS_API_KEY
                    }
        # print(url)
        
        response = requests.request("GET", url, headers=headers, data=payload)
        if response.status_code == 200:
            API_results.append(response.json())
        else:
            print(f"Error for coordinates {row}: {response.status_code}")
    return API_results
vancouver_FS_info = foursquare_info(lat_longs,radius)  
vancouver_FS_info


[{'results': [{'fsq_id': '4f60fc8be4b0a9d090a1f850',
    'categories': [{'id': 13003,
      'name': 'Bar',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
       'suffix': '.png'}},
     {'id': 13065,
      'name': 'Restaurant',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'}}],
    'distance': 250,
    'name': 'Rogue Kitchen & Wetbar',
    'popularity': 0.9823883325082823,
    'price': 3,
    'rating': 8.2},
   {'fsq_id': '51b201d27dd249ae714ba728',
    'categories': [{'id': 13003,
      'name': 'Bar',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
       'suffix': '.png'}},
     {'id': 13029,
      'name': 'Brewery',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/brewery_',
       'suffix': '.png'}},
     {'id': 13065,
      'name': 'Restaurant',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'su

In [46]:
import pandas as pd

vancouver_df = pd.json_normalize(vancouver_FS_info, errors='ignore')
vancouver_df['results']

#truncated post normalize?? 
## postive result = 51b201d27dd249ae714ba728
## negative result = 4aaa02cbf964a520905520e3 WHy?
vancouver_df
# vancouver_df.info()
# vancouver_df_normal = pd.json_normalize(vancouver_df)
# vancouver_df_normal
# vancouver_df[vancouver_df['fsq_id']=='51b201d27dd249ae714ba728']


,results,context.geo_bounds.circle.center.latitude,context.geo_bounds.circle.center.longitude,context.geo_bounds.circle.radius
0,"[{'fsq_id': '4f60fc8be4b0a9d090a1f850', 'categ...",49.262487,-123.114397,1000
1,"[{'fsq_id': '4b4fdbd2f964a520c91727e3', 'categ...",49.274566,-123.121817,1000
2,"[{'fsq_id': '4abea310f964a520ff8e20e3', 'categ...",49.279764,-123.110154,1000
3,"[{'fsq_id': '51b201d27dd249ae714ba728', 'categ...",49.260599,-123.113504,1000
4,"[{'fsq_id': '4f60fc8be4b0a9d090a1f850', 'categ...",49.264215,-123.117772,1000
...,...,...,...,...
240,"[{'fsq_id': '4e5060e7d22daf51d2848908', 'categ...",49.280977,-123.035969,1000
241,"[{'fsq_id': '4aa74c7bf964a520a34c20e3', 'categ...",49.270783,-123.141564,1000
242,"[{'fsq_id': '5978d136851de569bc454a8a', 'categ...",49.264019,-123.209176,1000
243,"[{'fsq_id': '509ad61591d477646f22cb2f', 'categ...",49.265800,-123.205960,1000


In [ ]:
### QUOC's Method ####

def foursquare(latitude, longitude, radius, api_key, categories):

##Construct the URL for the Foursquare API
    url = 'https://api.foursquare.com/v3/places/search'
    params = { "ll": f"{latitude},{longitude}", "radius": radius, "categoryId": categories,'fields': 'name,popularity,rating,price,distance,fsq_id'}

    headers = { "Accept": "application/json", "Authorization": api_key }
    API_Results = []
    headers ={
             "Accept": "application/json",
             'Authorization': FS_API_KEY
             }
    
    category_filter = "11062,13003,13004,13005,13006,13007,13008,13009,13010,13011,13012,13013,13014,13015,13016,13017,13018,13019,13020,13021,13022,13023,13024,13025,13059" 
    fields = "name,categories,popularity,rating,price,distance,fsq_id"
    params = f"&fields{fields}&categories{category_filter}"
    
    response = requests.get(url, params=params, headers=headers)

##Initialize an empty list to store the results
    
    if response.status_code != 200:
        return f"Error: {response.status_code}"
    else:
        data = response.json()
        for i in data['results']:
            details = {
            'name': i.get('name', None),
            'popularity': i.get('popularity', None),
            'rating': i.get('rating', None),
            'price': i.get('price', None),
            'distance': i.get('distance', None),
            'fsq_id': i.get('fsq_id', None)
        }
            API_Results.append(details)
        return API_Results

Consolidated_API_results = []  # Initialize empty list to accumulate results

##Loop latitude,longitude from station to get details
for index, row in vancouver_stations.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    station_id = row['id']
    result_list = foursquare(latitude, longitude, 1000, api_key, '4d4b7105d754a06374d81259,4bf58dd8d48988d116941735')

    for result in result_list:
        # Create a new dictionary with latitude and longitude, id of station, then update it with the result dictionary
        combined_result = {'latitude': latitude, 'longitude': longitude, 'id' : station_id}
        combined_result.update(result)

##Append the combined dictionary to 'all_results'
        all_results.append(combined_result)

##Convert the 'all_results' list of dictionaries to a DataFrame
df_foursquare = pd.DataFrame(all_results)
df_foursquare

# Importing the previous json from file:

In [ ]:
## Import a previously downloaded json file. 
import json
import pandas as pd
from pprint import pprint

with open ('../data/Foursquare_results.json') as data_file:
    vancouver_FS_info= json.load(data_file)
pprint(vancouver_FS_info)



In [ ]:
import pandas as pd

vancouver_df = pd.json_normalize(vancouver_FS_info, errors='ignore')
# vancouver_df.info()
jsonresults = vancouver_df['results'].explode()
jsonresults

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

NotImplementedError: 

Put your parsed results into a DataFrame

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [15]:
import requests
import os
YELP_API = os.getenv('YELP_API_KEY')
url = "https://api.yelp.com/v3/businesses/search?latitude=49.262487&longitude=-123.114397&radius=1000&categories=bars%2C%20restaurants&sort_by=best_match&limit=20"

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {YELP_API}"
}

response = requests.get(url, headers=headers)

print(response.text)

{"error": {"code": "VALIDATION_ERROR", "description": "'Bearer None' does not match '^(?i)Bearer [A-Za-z0-9\\\\-\\\\_]{128}$'", "field": "Authorization", "instance": "Bearer None"}}


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating